In [1]:
"""Help file to calculate errors and adjust the CSV data."""

import os

# scipy libs
import numpy as np
import pandas as pd

import pdb

In [2]:
def latex_err_fmt(val, err, figures=1):
    figures_adj = np.floor(np.log10(err)) - figures + 1
    adj_factor = 10 ** figures_adj
    
    err = np.round(err / adj_factor)
    val = np.round(val / adj_factor) * adj_factor
    
    return val, err



In [3]:
def V_error(V, vertical_position=0, div=1):
    """Calculates the error for voltages."""
    err_percent = 0.3 / 100
    V_adj = V + vertical_position
    div_err = .2 * div / 1000
    fix_err = 7 / 1000

    return err_percent * V_adj + div_err + fix_err


def t_error(t, div=None):
    """Error of time."""
    samples = len(t)

    if not div:
        div = (t[samples-1] - t[0]) / (samples - 1)

    total_t = samples * div
    err_frac = 50 / 1000000  # 50 ppm
    
    return np.zeros(samples) + total_t * err_frac



In [4]:
path = "dados/original/"
save_path = "dados/"

name_changes = {
    "Tempo (s)": "t",
    "V1 (V)": "V1",
    "V2 (V)": "V2"
}

horiz_pos = {
    "2ko": 500,
    "310o": 450,
    "50o": 450,
    "quad_deriv": 0,
    "quad_integ": 900,
    "ramp_deriv": 0, 
    "ramp_integ": 960,
    "sin_deriv": 0,
    "sin_integ": 900,
    "subcrit": 500,
    "supcrit": 500,
    "supcrit2": 500
}

for file in os.listdir(path):
    # get data and adjust it
    data = pd.read_csv(path + file, decimal=',', float_precision="high", dtype=np.longdouble)
    data = data.rename(index=str, columns=name_changes)
    
    for key in "t", "V1", "V2":
        data[key] = data[key] * 1000  # change to ms, mV

    # error calculations
    data["V1err"] = V_error(data["V1"], data["V1"].min())
    data["V2err"] = V_error(data["V2"], horiz_pos[file.replace(".csv", "")])
    data["terr"] = t_error(data["t"])

    # save new table for plotting
    data.to_csv(save_path + file)
    
    # LATEX: off now
    # # adjust error format for LaTeX's siunitx
    # for col in ("V1", "V2", "t"):
    #     data[col], data[col + "err"] = latex_err_fmt(data[col], data[col + "err"])
    #
    # print(data.to_latex())

    print("Done:", file)

Done: supcrit2.csv
Done: 310o.csv
Done: quad_integ.csv
Done: ramp_deriv.csv
Done: quad_deriv.csv
Done: ramp_integ.csv
Done: supcrit.csv
Done: sin_deriv.csv
Done: sin_integ.csv
Done: subcrit.csv
Done: 2ko.csv
Done: 50o.csv
